In [1]:
import pymupdf

# Ideas

**first:** transform to markdown\
**second:** split

In [2]:
def extract_articles(blocks) -> list[dict]:
    articles = []
    article = None
    article_flag = 0
    text = ""
    for block in blocks:
        #If the block starts with superscript text, it means it's a footnote
        if len(block["lines"]) > 0 and len(block["lines"][0]["spans"]) > 0 and block["lines"][0]["spans"][0]["size"] < 7:
            continue

        for line in block["lines"]:
            for span in line["spans"]:
                if article_flag == 1:
                    if span["font"] == "Arial-BoldMT":
                        if not (span["text"].startswith("Article") or span["text"].startswith("Rule")) and span["color"] == 0:
                            if not "title" in article:
                                article["title"] = ""
                            article["title"] += span["text"]
                    else:
                        if span["size"] > 7:
                            text += span["text"].rstrip("-")
                
                if span["font"] == "Arial-BoldMT" and (span["text"].startswith("Article") or span["text"].startswith("Rule")):
                    article_flag = 1
                    if article:
                        article["text"] = text
                        text = ""
                        if "title" in article:
                            article["title"] = article["title"].strip()
                        articles.append(article)
                    article = {}
                    article["id"] = span["text"].strip()
    if article:
        article["text"] = text
        if "title" in article:
            article["title"] = article["title"].strip()
        articles.append(article)

    return articles

In [13]:
def manage_overlap(page_blocks, section_origin: str):
    section_type = " ".join(section_origin.split(" ")[:2])
    sections_landmarks = {}
    for i in range(len(page_blocks)):
        if len(page_blocks[i]["lines"]) > 0 and len(page_blocks[i]["lines"][0]["spans"]) > 0 and page_blocks[i]["lines"][0]["spans"][0]["font"] == "Arial-BoldMT" and page_blocks[i]["lines"][0]["spans"][0]["color"] != 0:
            text = " ".join(page_blocks[i]["lines"][0]["spans"][0]["text"].strip().split(" ")[:2])
            sections_landmarks[text] = i
    if section_type not in sections_landmarks:
        if len(sections_landmarks) > 0:
            i_end = next(iter(sections_landmarks.values()))
            return page_blocks[:i_end]
        else:
            return page_blocks
    else:
        i_deb = sections_landmarks[section_type]
        section_crossed = False
        i_end = len(page_blocks)
        for elem in sections_landmarks:
            if section_crossed:
                i_end = sections_landmarks[elem]
            if elem == section_type:
                section_crossed = True
        return page_blocks[i_deb:i_end]
#algo en pseudo code 
#récupérer le nom du chapitre associé
#chercher dans la page les blocks de séparation autres que "article" ou "rule"
#récupérer les indices correspondant aux blocks entre deux blocs de séparation
#si la balise du chapitre correspond, on prend tout ce qu'il y a après et avant éventuelle prochaine balise
#sinon on prend tout ce qu'il y a après et jusqu'à éventuelle prochaine balise

In [14]:
path = "../resources/Datasets - v1/Official Legal Publications/1-EPC_17th_edition_2020_en.pdf"
blocks = []
begin_section = 352
end_section = 352
with pymupdf.open(path) as pdf:
    for pn in range(begin_section-1, end_section):
        page = pdf[pn]
        print(page.number)
        height, width = page.rect.height, page.rect.width
        print(height, width)
        page.set_cropbox((width/2, 0, width, height-40))
        if pn == begin_section-1:
            b = manage_overlap(page, "Chapter IV Certificate of exhibition")
            print(b)
            blocks.extend(b)
        elif pn == end_section-1:
            blocks.extend(manage_overlap(page, "Chapter IV rien a foutre"))
        else:
            blocks.extend(page.get_text("dict", flags=11)["blocks"])
        print(blocks)
articles = extract_articles(blocks)

351
666.239990234375 467.760009765625


TypeError: object of type 'Page' has no len()

In [15]:
import pandas as pd
df = pd.DataFrame(articles)
df[df["id"] == "Rule 25"]["text"].item()

NameError: name 'articles' is not defined

In [16]:
path = "../resources/Datasets - v1/Official Legal Publications/1-EPC_17th_edition_2020_en.pdf"
content = []
with pymupdf.open(path) as pdf:
    height, width = pdf[8].rect.height, pdf[8].rect.width
    toc = pdf.get_toc()
    for level, title, page_num in toc:
        indent = "    " * (level - 1)  # 4 espaces par niveau de profondeur
        print(f"{indent}- {title} (Page {page_num})")
    
    levels = ["", "", "", ""]
    last_level = 0
    section_flag = 0
    for level, title, page_num in toc:
        if level < last_level:
            if section_flag == 1:
                section_flag = 0
                blocks = []
                for pn in range(begin_section-1, page_num):
                    
                    page = pdf[pn]
                    page.set_cropbox((width/2, 0, width, height-40))
                    page_blocks = page.get_text("dict", flags=11)["blocks"]
                    section_origin = list(filter(lambda x: x != "", levels))[-1]
                    if pn == begin_section-1:
                        blocks.extend(manage_overlap(page_blocks, section_origin))
                    elif pn == page_num-1:
                        blocks.extend(manage_overlap(page_blocks, section_origin))
                    else:
                        blocks.extend(page_blocks)
                
                section_articles = extract_articles(blocks)
                for article in section_articles:
                    article["origin"] = " -> ".join(list(filter(lambda x: x != "", levels)))
                content.extend(section_articles)

            for i in range(level, len(levels)):
                levels[i] = ""
        

        if title.startswith("Article") or title.startswith("Rule"):
            if level > last_level:
                begin_section = page_num
                section_flag = 1

        else:
            levels[level-1] = title

        last_level = level

- Convention on the Grant of European Patents (European Patent Convention) (Page 7)
    - FOREWORD (Page 8)
        - Citation practice (Page 10)
        - New text of the EPC (Page 12)
        - Transitional provisions (Page 14)
        - Crossreferences (Page 14)
- Convention on the Grant of European Patents (European Patent Convention) (Page 21)
    - PREAMBLE (Page 46)
    - PART I GENERAL AND INSTITUTIONAL PROVISIONS (Page 48)
        - Chapter I General provisions (Page 48)
            - Article 1 European law for the grant of patents (Page 48)
            - Article 2 European patent (Page 48)
            - Article 3 Territorial effect (Page 48)
            - Article 4 European Patent Organisation (Page 50)
            - Article 4a Conference of ministers of the Contracting States (Page 50)
        - Chapter II The European Patent Organisation (Page 50)
            - Article 5 Legal status (Page 50)
            - Article 6 Headquarters (Page 52)
            - Article 7 Suboffices

In [17]:
import pandas as pd
df = pd.DataFrame(content)
df[df["id"] == "Rule 25"]["text"].item()

'Within four months of filing the European patent application, the applicant shall file the certificate referred to in Article 55, paragraph 2, which: (a) is issued at the exhibition by the authority responsible for the protection of industrial property at that exhibition; (b) states that the invention was in fact displayed there; (c) states the opening date of the exhibition and, where the invention was disclosed later than on that date, the date on which the invention was first disclosed; and  (d) is accompanied by an identification of the invention, duly authenticated by the above-mentioned authority. '

In [18]:
df.to_csv("../resources/EPC_articles.csv")

In [4]:
path = "../resources/Datasets - v1/Official Legal Publications/1-EPC_17th_edition_2020_en.pdf"
with pymupdf.open(path) as pdf:
    page = pdf[347]
    height, width = page.rect.height, page.rect.width
    page.set_cropbox((width/2, 0, width, height))
    for block in page.get_text("dict", flags=11)["blocks"]:
        #If the block starts with superscript text, it means it's a footnote
        if block["lines"][0]["spans"][0]["size"] < 7:
            continue
        
        for line in block["lines"]:
            for span in line["spans"]:
                print(f"Font: {span['font']}\nText: {span['text']}\nSize: {span['size']}\nBbox: {span['bbox']}\ncolor: {span['color']}\n-------------------")
            print("----- EOL -----")
        print("----- EOBlock -----")

Font: Arial-BoldMT
Text: Rule 21
Size: 8.039999961853027
Bbox: (7.0800018310546875, 64.17526245117188, 35.669490814208984, 75.38302612304688)
color: 0
-------------------
Font: ArialMT
Text: 29
Size: 5.039999961853027
Bbox: (35.639984130859375, 64.28974914550781, 41.20415496826172, 70.99798583984375)
color: 0
-------------------
Font: Arial-BoldMT
Text:   
Size: 8.039999961853027
Bbox: (41.160003662109375, 64.17526245117188, 45.55512619018555, 75.38302612304688)
color: 0
-------------------
----- EOL -----
Font: Arial-BoldMT
Text: Rectification of the designation of an 
Size: 8.039999961853027
Bbox: (7.0800018310546875, 74.85525512695312, 149.82557678222656, 86.06301879882812)
color: 0
-------------------
----- EOL -----
Font: Arial-BoldMT
Text: inventor 
Size: 8.039999961853027
Bbox: (7.0800018310546875, 85.53524780273438, 40.87510299682617, 96.74301147460938)
color: 0
-------------------
----- EOL -----
----- EOBlock -----
Font: ArialMT
Text: (1) 
Size: 8.039999961853027
Bbox: (7.080